# Michał Matak - sieć neuronowa do mapowania obiektu



### Importowanie bibliotek
* numpy - operacje na macierzach 
* Nwt - sieć neuronowa własnej implementacji

In [2]:
import numpy as np
from Net import Net
import matplotlib.pyplot as plt
import csv
import random

### Funkcje przekształcające strukturę zbioru danych

Zakodowanie pomiarów w macierz:

In [3]:
data_dictionary = {
    "13.0_10.0":1,
    "14.0_10.0":2,
    "15.0_10.0":3,
    "15.0_11.0":4,
    "15.0_12.0":5,
    "20.0_5.0":6
}

In [4]:
data_dictionary = {
    "14.0_10.0":1,
    "15.0_10.0":2,
    "15.0_11.0":3,
}

In [5]:
print(data_dictionary)

{'14.0_10.0': 1, '15.0_10.0': 2, '15.0_11.0': 3}


In [6]:
def encode(number):
    vector = np.zeros(len(data_dictionary))
    vector[number-1] = 1
    return vector

In [7]:
def decode(vector):
    return np.argmax(vector) + 1

In [8]:
def classify(array):
    return np.array([1 if element == max(array) else 0 for element in array], dtype="float64")

In [9]:
x = encode(2)
print(x)
print(decode(x))

[0. 1. 0.]
2


Pobranie danych z pliku:

In [10]:
def control(reading):
    control = 50
    return min(reading, control)/control

In [11]:
def get_data(measurement):
    offset = 0
    sample_data = []
    with open("standarized/" + measurement + ".txt", newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if len(row) == 6 + offset:
                odczyt1 = float(row[1+offset])
                odczyt2 = float(row[3+offset])
                odczyt3 = float(row[5+offset])
                sample_data.append([np.array([control(odczyt1), control(odczyt2), control(odczyt3)]),
                                    encode(data_dictionary[measurement])])
    return sample_data

Rezultatem jest lista **data** składająca się z list w formie: \[ \[atrybuty\], \[macierz odpowiadająca gatunkowi\] \]

Podział danych na zbiór testowy według **train_test_ratio**:

In [12]:
def create_sets(data, set1_set2_ratio):
    np.random.shuffle(data)
    set1_size = int(set1_set2_ratio*len(data))
    set2_size = len(data) - set1_size
    set1 = data[:set1_size]
    set2 = data[-set2_size:]
    return set1, set2

In [13]:
data = []
for element in data_dictionary:
    data += get_data(element)
data = np.array(data)
print(len(data))

3000


In [39]:
train_set, test_set = create_sets(data, 0.75)

### Sieć neuronowa

In [40]:
len(data_dictionary)

3

In [41]:
train_set, test_set = create_sets(data, 0.25)

In [42]:
NN = Net([3, 200, 100, 10, len(data_dictionary)], False, 0.1)

In [43]:
x = np.array(train_set)[:, 0]
y = np.array(train_set)[:, 1]

In [44]:
NN.train(x, y, 100000, True)

In [45]:
print(NN(test_set[0][0]))
print(classify(NN(test_set[0][0])))
print(decode(classify(NN(test_set[0][0]))))
print(decode(test_set[0][1]))

[0.39357081 0.52098572 0.12482708]
[0. 1. 0.]
2
1


### Ewaluacja - sprawdzenie działania algorytmu

In [46]:
errors = 0
for element in test_set:
    if decode(classify(NN(element[0]))) != decode(element[1]):
        errors +=1
error_rate = errors/len(test_set)
print(f"Accuracy: {(1-error_rate)*100:.02f}%")

Accuracy: 60.89%
